In [1]:
pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install python-mnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install emnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#this is the same code we used for a CNN predicting a number based on a handmade drawing of it
#for this, i forked another repository: https://github.com/JosePabloGFnl/exportacion-numeros

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from mnist import MNIST
from emnist import extract_training_samples
from emnist import list_datasets
from emnist import extract_test_samples
import numpy as np

#EMNIST data set entrenamiento
datos_entrenamiento, datos_etiquetas = extract_training_samples('byclass')
imagenes_de_prueba, etiquetas_de_prueba = extract_test_samples('byclass')

datos_entrenamiento = tf.keras.utils.normalize(datos_entrenamiento, axis = 1)
datos_entrenamiento = np.expand_dims(datos_entrenamiento, axis=3)
imagenes_de_prueba = tf.keras.utils.normalize(imagenes_de_prueba, axis = 1)
imagenes_de_prueba = np.expand_dims(imagenes_de_prueba, axis=3)

#fit al generador usando tensorflow
rotation_range_val = 15
width_shift_val = 0.10
height_shift_val = 0.10
gen_entrenamiento = ImageDataGenerator(rotation_range = rotation_range_val,
                             width_shift_range = width_shift_val,
                             height_shift_range = height_shift_val)
gen_entrenamiento.fit(datos_entrenamiento.reshape(datos_entrenamiento.shape[0], 28, 28, 1))

#datos, metadatos = tfds.load('emnist', as_supervised=True, with_info=True)

#datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

#nombres_clases = metadatos.features['label'].names

#normalizing function, makes it faster
#def normalizar(imagenes, etiquetas):
#  imagenes = tf.cast(imagenes, tf.float32)
#  imagenes /= 255 #0-255 to 0-1
#  return imagenes, etiquetas

#datos_entrenamiento = datos_entrenamiento.map(normalizar)
#datos_pruebas = datos_pruebas.map(normalizar)

#datos_entrenamiento = datos_entrenamiento.cache()
#datos_pruebas = datos_pruebas.cache()

#clases = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [5]:
#import matplotlib.pyplot as plt
#plt.figure(figsize=(10,10))

#for i, (imagen, etiqueta) in enumerate(datos_entrenamiento.take(25)):
#  imagen = imagen.numpy().reshape((28,28))
#  plt.subplot(5,5, i +1)
#  plt.xticks([])
#  plt.yticks([])
#  plt.grid(False)
#  plt.imshow(imagen, cmap=plt.cm.binary)
#  plt.xlabel(clases[etiqueta])
#plt.show()

In [6]:
#creating the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32,
                kernel_size=(3, 3),
                input_shape=(28, 28, 1),
                activation='relu',
                padding='same'))
model.add(tf.keras.layers.Conv2D(64,
                kernel_size=(3, 3),
                activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(128, 
                                 (3,3), 
                                 activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(62, activation=tf.nn.softmax))

#Compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

#num_ej_entrenamiento = metadatos.splits["train"].num_examples
#num_ej_pruebas = metadatos.splits["test"].num_examples

TAMANO_LOTE = 1024

#shuffling
#datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
#datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

import math

datos_val = ImageDataGenerator()
datos_val.fit(imagenes_de_prueba.reshape(imagenes_de_prueba.shape[0], 28, 28, 1))

#training
historial = model.fit(gen_entrenamiento.flow(datos_entrenamiento, datos_etiquetas, batch_size=TAMANO_LOTE), epochs=20, validation_data = datos_val.flow(imagenes_de_prueba, etiquetas_de_prueba, batch_size=32))

Epoch 1/20
682/682 [==============================] - 171s 234ms/step - loss: 0.9097 - accuracy: 0.7294 - val_loss: 0.4579 - val_accuracy: 0.8386
Epoch 2/20
682/682 [==============================] - 161s 236ms/step - loss: 0.5057 - accuracy: 0.8240 - val_loss: 0.4082 - val_accuracy: 0.8519
Epoch 3/20
682/682 [==============================] - 159s 233ms/step - loss: 0.4564 - accuracy: 0.8373 - val_loss: 0.3820 - val_accuracy: 0.8598
Epoch 4/20
682/682 [==============================] - 159s 234ms/step - loss: 0.4326 - accuracy: 0.8442 - val_loss: 0.3773 - val_accuracy: 0.8576
Epoch 5/20
682/682 [==============================] - 160s 234ms/step - loss: 0.4174 - accuracy: 0.8482 - val_loss: 0.3676 - val_accuracy: 0.8647
Epoch 6/20
682/682 [==============================] - 158s 232ms/step - loss: 0.4060 - accuracy: 0.8518 - val_loss: 0.3671 - val_accuracy: 0.8629
Epoch 7/20
682/682 [==============================] - 160s 234ms/step - loss: 0.3973 - accuracy: 0.8544 - val_loss: 0.3563 -

In [7]:
#exporting the model to h5
model.save('numeros.h5')

In [8]:
!ls

numeros.h5  sample_data


In [9]:
!pip install tensorflow js

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 52.3 MB/s 
  Created wheel for js: filename=js-1.0-py3-none-any.whl size=2907 sha256=cf6c31f42d14ea353033741780324bf506b015e6f69ebfe250f4e6463c135d71
  Stored in directory: /root/.cache/pip/wheels/bf/92/56/6597c8f32980ee64af7e864796e8d7423eb67eafff5415a4ae
  Created wheel for shutilwhich: filename=shutilwhich-1.1.0-py3-none-any.whl size=2781 sha256=932d70f789706f2536b62f1a9f5731003d0692b2cb540d4e13f90a51574960e1
  Stored in directory: /root/.cache/pip/wheels/4c/c0/ce/5a47f35186acbe73e38811df30daaf798c81a7599bb2d5995a
Successfully built js shutilwhich


In [10]:
#folder with output files
!mkdir carpeta_salida

In [14]:
#export to output folder
!tensorflowjs_converter --input_format keras numeros.h5 carpeta_salida

In [12]:
!ls carpeta_salida

group1-shard1of1.bin  model.json


In [13]:
#note, if tensorflow does not work in the page, do the following:
#open cmd > cd [folder] > python -m http.server 8000
#in the address, localhost:8000/index.html